# TEAM : <font color='red'> ONE_MAN_ARMY </font>

---
---

Please run all cell after changing anything or keep track of <font color='blue'>current directories 


# checking gpu status


In [1]:
import torch
from IPython.display import Image, clear_output
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

Using device: cuda

Tesla T4
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


# Downloding Yolov5 from official repo 
   I'm downloading the repo everytime I do inference , output may change a little as repo gets updated, but i dont think that will cause any problem 

In [2]:
!git clone https://github.com/ultralytics/yolov5
!pip install -qr yolov5/requirements.txt
%cd yolov5

import torch
from IPython.display import Image, clear_output
from utils.google_utils import gdrive_download

clear_output()

# configuration 
* class information has to be stored in  <font color="blue"> _data.yaml_ </font> 
* configure the model custom <font color="green"> yolov5m </font> in <font color="blue"> _custom_yolov5m.yaml_ </font>  

<a id=""></a>
# Download Test set
2nd rounds test set \
Directories should look like this:

![DIR.PNG](https://drive.google.com/uc?export=view&id=1JgbJJzaNGJ7Cy6B1pbd4p7qP4PuuP2mU) 
![DIR.PNG](https://drive.google.com/uc?export=view&id=12Kmq7KS5VXJlgg9-ZPIyR1zxz7uMBF2E)

In [3]:
%cd /content
!gdown --id 1XlDCa4rYI84uYSKAPVuigRcKvJRxcsh8   #change the drive link id for new test set
!unrar x test.rar                      
!mv 'test round 2/' 'test'                       # unzip or do anything to save test set in /content/test/{all images}
clear_output()

# Download weights

In [3]:
!gdown --id 1v_m6BsOdKvXmofp_5Ds6snGdjBk6oXNL #bestdayv5.pt
!gdown --id 1-LyZsp3u_BjYTesMi05yeRndfXuRJcz1 #bestdayv52.pt                             
!gdown --id 1-o6wDRCkP1k_Z5tMrg3sFKcssD2LQzRP #bestdayv53.pt
!gdown --id 1D0vlLBIEdJ9MJskuCRKnh5Lta67O0tZt #bestnightr2v5.pt

Downloading...
From: https://drive.google.com/uc?id=1v_m6BsOdKvXmofp_5Ds6snGdjBk6oXNL
To: /content/yolov5/bestdayv5.pt
173MB [00:02, 76.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-LyZsp3u_BjYTesMi05yeRndfXuRJcz1
To: /content/yolov5/bestdayv52.pt
43.6MB [00:01, 42.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-o6wDRCkP1k_Z5tMrg3sFKcssD2LQzRP
To: /content/yolov5/bestdayv53.pt
43.6MB [00:01, 40.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1D0vlLBIEdJ9MJskuCRKnh5Lta67O0tZt
To: /content/yolov5/bestnightr2v5.pt
43.6MB [00:01, 39.1MB/s]


# Detection

In [19]:
%cd /content/yolov5/
import argparse
import time
from pathlib import Path

import cv2
import torch
import torch.backends.cudnn as cudnn
from numpy import random

from models.experimental import attempt_load
from utils.datasets import LoadStreams, LoadImages
from utils.general import check_img_size, non_max_suppression, apply_classifier, scale_coords, xyxy2xywh, \
    strip_optimizer, set_logging, increment_path
from utils.plots import plot_one_box
from utils.torch_utils import select_device, load_classifier, time_synchronized


def detect(save_img=True):
    source, weights, view_img, save_txt, imgsz = opt.source, opt.weights, opt.view_img, opt.save_txt, opt.img_size
    webcam = source.isnumeric() or source.endswith('.txt') or source.lower().startswith(
        ('rtsp://', 'rtmp://', 'http://'))

    # Directories
    save_dir = Path(increment_path(Path(opt.project) / opt.name, exist_ok=opt.exist_ok))  # increment run
    (save_dir / 'labels' if save_txt else save_dir).mkdir(parents=True, exist_ok=True)  # make dir

    # Initialize
    set_logging()
    device = select_device(opt.device)
    half = device.type != 'cpu'  # half precision only supported on CUDA

    # Load model
    model = attempt_load(weights, map_location=device)  # load FP32 model
    imgsz = check_img_size(imgsz, s=model.stride.max())  # check img_size
    if half:
        model.half()  # to FP16

    # Second-stage classifier
    classify = False
    if classify:
        modelc = load_classifier(name='resnet101', n=2)  # initialize
        modelc.load_state_dict(torch.load('weights/resnet101.pt', map_location=device)['model']).to(device).eval()

    # Set Dataloader
    vid_path, vid_writer = None, None
    if webcam:
        view_img = True
        cudnn.benchmark = True  # set True to speed up constant image size inference
        dataset = LoadStreams(source, img_size=imgsz)
    else:
        save_img = True
        dataset = LoadImages(source, img_size=imgsz)

    # Get names and colors
    names = model.module.names if hasattr(model, 'module') else model.names
    colors = [[random.randint(0, 255) for _ in range(3)] for _ in names]

    # Run inference
    t0 = time.time()
    img = torch.zeros((1, 3, imgsz, imgsz), device=device)  # init img
    _ = model(img.half() if half else img) if device.type != 'cpu' else None  # run once
    results=[]
    for path, img, im0s, vid_cap in dataset:
        img = torch.from_numpy(img).to(device)
        img = img.half() if half else img.float()  # uint8 to fp16/32
        img /= 255.0  # 0 - 255 to 0.0 - 1.0
        if img.ndimension() == 3:
            img = img.unsqueeze(0)

        # Inference
        t1 = time_synchronized()
        pred = model(img, augment=opt.augment)[0]

        # Apply NMS
        pred = non_max_suppression(pred, opt.conf_thres, opt.iou_thres, classes=opt.classes, agnostic=opt.agnostic_nms)
        t2 = time_synchronized()

        # Apply Classifier
        if classify:
            pred = apply_classifier(pred, modelc, img, im0s)

        # Process detections
        for i, det in enumerate(pred):  # detections per image
            if webcam:  # batch_size >= 1
                p, s, im0 = Path(path[i]), '%g: ' % i, im0s[i].copy()
            else:
                p, s, im0 = Path(path), '', im0s

            save_path = str(save_dir / p.name)
            txt_path = str(save_dir / 'labels' / p.stem) + ('_%g' % dataset.frame if dataset.mode == 'video' else '')
            s += '%gx%g ' % img.shape[2:]  # print string
            gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  # normalization gain whwh
            if len(det):
                # Rescale boxes from img_size to im0 size
                det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0.shape).round()

                # Print results
                for c in det[:, -1].unique():
                    n = (det[:, -1] == c).sum()  # detections per class
                    s += '%g %ss, ' % (n, names[int(c)])  # add to string
                
                xmin = []
                ymin = []
                xmax = []
                ymax = []
                scores = []
                labels_value=[]
                image_ids=[]
                # Wr ite results
                for *xyxy, conf, cls in reversed(det):
                    if not save_txt:  # Write to file

                        conf_score = '%.2f' % (conf)
                        label_with_cls = '%s' % (names[int(cls)])
                        
                        labels_value.append(label_with_cls)
                        
                        xmin.append(int(xyxy[0]))
                        ymin.append(int(xyxy[1]))
                        xmax.append(int(xyxy[2]))
                        ymax.append(int(xyxy[3]))
                        
                        scores.append(conf_score)
                        image_ids.append(save_path)

                       

                    
                    label = '%s %.2f' % (names[int(cls)], conf)
                    plot_one_box(xyxy, im0, label=label, color=colors[int(cls)], line_thickness=3)

            # Print time (inference + NMS)
            print('%sDone. (%.3fs)' % (s, t2 - t1))

          
        result = {
            'image_id': image_ids,
            'class': labels_value,
            'score': scores,
            'xmin': xmin,
            'ymin': ymin,
            'xmax': xmax,
            'ymax': ymax
            }  
        results.append(result)
    if save_txt or save_img:
        s = f"\n{len(list(save_dir.glob('labels/*.txt')))} labels saved to {save_dir / 'labels'}" if save_txt else ''
        print(f"Results saved to {save_dir}{s}")

    print('Done. (%.3fs)' % (time.time() - t0))

    return results


/content/yolov5


# Now we are inside Yolov5 directory

In [26]:
!python detect.py --img-size 2048 --weight '../bestdayv53.pt'  --augment

Namespace(agnostic_nms=False, augment=True, classes=None, conf_thres=0.25, device='', exist_ok=False, img_size=2048, iou_thres=0.45, name='exp', project='runs/detect', save_conf=False, save_txt=False, source='data/images', update=False, view_img=False, weights=['../bestdayv53.pt'])
Using torch 1.7.0+cu101 CUDA:0 (Tesla T4, 15079MB)

Fusing layers... 
Model Summary: 316 layers, 21549450 parameters, 0 gradients, 51.5 GFLOPS
image 1/1 /content/yolov5/data/images/maxresdefault.jpg: 1152x2048 1 ambulances, 8 buss, 15 cars, 1 minivans, 2 motorbikes, 1 pickups, 1 suvs, 4 three wheelers (CNG)s, 1 vans, Done. (0.116s)
Results saved to runs/detect/exp20
Done. (0.304s)


In [20]:
class opt:
      weights=['../bestdayv5.pt','../bestdayv52.pt','../bestdayv53.pt']
      source='../*.jpg'
      save_txt=False
      output='../outputday'  # Output directory of the results
      classes=False
      img_size=1344    # Inference Imag Size
      conf_thres=0.3
      iou_thres=0.55
      fourcc='mp4v'
      half=False
      device=''
      view_img=False
      agnostic_nms=False
      augment=True
      project='yolov5_traffic'
      name='dhakaAi'
      exist_ok=False
with torch.no_grad():
   
      res1=detect()

class opt:
      weights='../bestnightr2v5.pt'
      source='../*.jpg'
      save_txt=False
      output='../outputnight'  # Output directory of the results
      classes=False
      img_size=1344    # Inference Imag Size
      conf_thres=0.3
      iou_thres=0.55
      fourcc='mp4v'
      half=False
      device=''
      view_img=False
      agnostic_nms=False
      augment=True
      project='yolov5_traffic'
      name='dhakaAi'
      exist_ok=False
with torch.no_grad():
    
      res2=detect()


Using torch 1.7.0+cu101 CUDA:0 (Tesla T4, 15079MB)



Fusing layers... 


Model Summary: 316 layers, 21549450 parameters, 0 gradients, 51.5 GFLOPS


Fusing layers... 


Model Summary: 316 layers, 21549450 parameters, 0 gradients, 51.5 GFLOPS


...

Ensemble created with ['../bestdayv5.pt', '../bestdayv52.pt', '../bestdayv53.pt']

image 1/1 /content/gettyimages-652959585-612x612.jpg: 896x1344 2 buss, 7 cars, 1 pickups, 1 trucks, Done. (0.187s)
Results saved to yolov5_traffic/dhakaAi12
Done. (0.389s)


Using torch 1.7.0+cu101 CUDA:0 (Tesla T4, 15079MB)



Fusing layers... 


Model Summary: 316 layers, 21549450 parameters, 0 gradients, 51.5 GFLOPS


image 1/1 /content/gettyimages-652959585-612x612.jpg: 896x1344 2 buss, 3 cars, 1 motorbikes, 5 pickups, Done. (0.060s)
Results saved to yolov5_traffic/dhakaAi13
Done. (0.110s)


# Submission file generation

Two initial submission files will be generated generated inside directory /conten/yolov5 
 

In [7]:
import pandas as pd
import numpy as np

append_data1=[]
append_data2=[]

for i in range(len(res1)):
  df = pd.DataFrame(res1[i], columns = ['image_id','class','score','xmin','ymin','xmax','ymax'])
  append_data1.append(df)
for i in range(len(res2)):
  df = pd.DataFrame(res2[i], columns = ['image_id','class','score','xmin','ymin','xmax','ymax'])
  append_data2.append(df)

finl_results1=pd.concat(append_data1)
finl_results1.image_id = [x.split('/')[-1] for x in finl_results1.image_id]

finl_results2=pd.concat(append_data2)
finl_results2.image_id = [x.split('/')[-1] for x in finl_results2.image_id]


finl_results1['width'] = 1024
finl_results1['height'] = 1024

finl_results2['width'] = 1024
finl_results2['height'] = 1024

print(f'Number of Objects Detected in day : {finl_results1.shape[0]}')
print(f'Number of Objects Detected in day : {finl_results2.shape[0]}')


finl_results1.to_csv(f'day_submission.csv', index=False)
finl_results2.to_csv(f'night_submission.csv', index=False)


Number of Objects Detected in day : 2978
Number of Objects Detected in day : 2560


# Ensembled submission  
* We are using  _day_submission_ and <font color='green'> _night_submission_</font> and <font color='green'> _day_submission_</font> for ensembling
* Ensembling will done based on <font color='blue'>Brightness of Test image


In [8]:
def avgBrightness(img_path):
    rgb_image=cv2.imread(img_path)
    
    hsv = cv2.cvtColor(rgb_image, cv2.COLOR_BGR2HSV)
    sum_brightness = np.sum(hsv[:,:,2])
    area = hsv.shape[0]*hsv.shape[1] # pixels
    avg = sum_brightness/area
    return avg


In [10]:
import torchvision
!rm -r /content/output
!mkdir /content/output
def ensemble():
  label_map ={
     "ambulance": 0,
     "auto rickshaw": 1,
     "bicycle": 2,
     "bus": 3,
     "car": 4,
     "garbagevan": 5,
     "human hauler": 6,
     "minibus": 7,
     "minivan": 8,
     "motorbike": 9,
     "pickup": 10,
     "army vehicle": 11,
     "policecar": 12,
     "rickshaw": 13,
     "scooter": 14,
     "suv": 15,
     "taxi": 16,
     "three wheelers (CNG)": 17,
     "truck": 18,
     "van": 19,
     "wheelbarrow": 20
     }
  rev_label_map = {v: k for k, v in label_map.items()}
  daycsv=pd.read_csv('/content/yolov5/day_submission.csv')
  nightcsv=pd.read_csv('/content/yolov5/night_submission.csv')
  allcsv = pd.concat([daycsv,nightcsv])
  allcsv=allcsv.sort_values(by=['image_id']) 
  pd.set_option('display.expand_frame_repr', False)
  #print(allcsv.head(15))
  #print(allcsv.info())
  
  image_ids=allcsv['image_id'].unique()
  
  final=pd.DataFrame(columns=['image_id','class','score','xmin','ymin','xmax','ymax','width','height'])
  for id in image_ids:

    print(id)
    
    img_path='/content/test/'+id
    brightness=avgBrightness(img_path)
    if brightness<70:
      
      view=allcsv.loc[allcsv['image_id']==id]
      #print(view.info())
      det=view[['xmin','ymin','xmax','ymax']].to_numpy()
      det.astype("float32")
      sc=view['score'].to_numpy()
      sc.astype("float32")
      cls_name=view['class'].to_list()
      cls=[]
      for c in cls_name:
        cls.append(label_map[c])

      cls=np.array(cls)

      sc=torch.FloatTensor(sc)
      det=torch.FloatTensor(det)

      idx=torchvision.ops.nms(det,sc,.55)
      det,sc,cls=det[idx].numpy(),sc[idx].numpy(),cls[idx]

      print(id)
      omni=np.insert(det,4,sc,axis=1)
      omni=np.insert(omni,5,cls,axis=1)   
      img=cv2.imread('/content/test'+id)
      for d in omni:
        final=final.append({'image_id':id,'class':rev_label_map[int(d[5])],'score':d[4],'xmin':d[0],'ymin':d[1],'xmax':d[2],'ymax':d[3],'width':1024,'height':1024},ignore_index=True)
        
        print('                          '+rev_label_map[int(d[5])]+'   score:'+str(d[4])+'      pos:' +str(d[0:4])) 

      final=final.append(nightcsv.loc[nightcsv["image_id"]==id])
      print("       night           ")
    else:
      img=cv2.imread('/content/test'+id)
      final=final.append(daycsv.loc[daycsv["image_id"]==id])
      print("       day             ")
    #print(det)
    #print(cls)
    #print(sc)
  return final
     
final=ensemble()
final.info()

Asraf_100_jpg.rf.472d5bd950587612520051eebd827328.jpg
       day             
Asraf_101_jpg.rf.58690ebe018b849e3d3f6ef8785ca148.jpg
       day             
Asraf_102_jpg.rf.40d695ed6f8db102530afcb11db466fa.jpg
       day             
Asraf_103_jpg.rf.19874eeb1f0a60c783cf34375a7c724b.jpg
       day             
Asraf_104_jpg.rf.a5288ec7a648c79f1806a0df2cddeb55.jpg
       day             
Asraf_105_jpg.rf.c632d88cde53186ccf67d13ef8abcc78.jpg
       day             
Asraf_106_jpg.rf.7992ce0a5986190ec8edc556b2e777b4.jpg
       day             
Asraf_107_jpg.rf.d07a632c065fd4f81de21c0fe7f4d0b9.jpg
       day             
Asraf_108_jpg.rf.6141d24a0c99a85b5dd171596e155f4c.jpg
       day             
Asraf_109_jpg.rf.1d9c0aa1826587208ad51ac330e78611.jpg
       day             
Asraf_110_jpg.rf.1db956dfe524cd925629f615131137ac.jpg
       day             
Asraf_111_jpg.rf.c063660bbf498d06b785325c68085c2e.jpg
       day             
Asraf_64_jpg.rf.779cb456e69c0805fa0a50bc39a9461f.jpg
       day 

# Final submission file

In [ ]:
%cd /content/
final.to_csv('submission.csv',index=False)